<a href="https://colab.research.google.com/github/HarryPotter12/PractiseML/blob/master/AML_Hackathon_Codon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# !pip install kaggle --upgrade

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command
!rm *.csv

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle competitions download -c iith-aml-hackathon-2020

  0% 0.00/1.06M [00:00<?, ?B/s]
100% 1.06M/1.06M [00:00<00:00, 31.0MB/s]
  0% 0.00/2.91M [00:00<?, ?B/s]
100% 2.91M/2.91M [00:00<00:00, 90.1MB/s]


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Archive:  Test.csv.zip
  inflating: Test.csv                

Archive:  Train.csv.zip
  inflating: Train.csv               

2 archives were successfully processed.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv("/content/gdrive/My Drive/Kaggle/Train.csv")
df_test = pd.read_csv("/content/gdrive/My Drive/Kaggle/Test.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train['Kingdom'].value_counts()

In [ ]:
df_test.info()

In [ ]:
X_train_full = df_train.loc[:, ~df_train.columns.isin(['Kingdom'])].values
y_train_full = df_train.loc[:, df_train.columns == 'Kingdom'].values

Here, I label-encoded the target variable.

In [ ]:
labelencoder = LabelEncoder()
y_train_full = labelencoder.fit_transform(y_train_full.ravel())
# y_train_full = np_utils.to_categorical(y_train_full)

Divided the training data in Training and Validation set

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size = 0.25, random_state = 42, stratify = y_train_full)

In [ ]:
print(X_train.shape)
print(y_train.shape)

(3908, 65)
(3908,)


In [ ]:
print(X_train_full.shape)

(5211, 65)


In [ ]:
X_test = df_test.loc[:, ~df_test.columns.isin(['Kingdom','Id'])].values

In [ ]:
X_test.shape

(7817, 65)

Let's scale the features to bring to same scale. We fit the scaler on training data and later scaled the validation data.

In [ ]:
# Feature Scaling to bring the variable in a single scale
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

In [ ]:
# # Tried this and didn't get good result, so, commented off
# # Feature Scaling to bring the variable in a single scale
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler()
# X_train = sc.fit_transform(X_train)
# X_val = sc.transform(X_val)

Let's create a few baseline models and see the performance. \
I One-Hot-Encoded (i.e. changed as categories and split into number of features) the target variable for a few models. But as my final model didn't need this I kept those commented.

In [ ]:
# # Commented as it didn't perform well
# from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression(max_iter=1000, random_state = 0)
# classifier.fit(X_train, y_train.ravel())
# y_val_pred = classifier.predict(X_val)

In [ ]:
# It performed pretty well
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
y_val_pred = classifier.predict(X_val)

In [ ]:
# # Commented as it didn't perform well
# from sklearn.svm import SVC
# classifier = SVC(kernel = 'linear', random_state = 0)
# classifier.fit(X_train, y_train.ravel())
# y_val_pred = classifier.predict(X_val)

In [ ]:
# It performed pretty well
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0, C=10, gamma=0.01)
classifier.fit(X_train, y_train.ravel())
y_val_pred = classifier.predict(X_val)

In [ ]:
# It performed pretty well
from sklearn.svm import NuSVC
classifier = NuSVC(kernel = "rbf", nu = 0.007, probability = True)
classifier.fit(X_train, y_train.ravel())
y_val_pred = classifier.predict(X_val)

In [ ]:
# # Commented as it didn't perform well
# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# classifier.fit(X_train, y_train.ravel())
# y_val_pred = classifier.predict(X_val)

In [ ]:
# # Commented as it didn't perform well
# from sklearn.tree import DecisionTreeClassifier
# classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
# classifier.fit(X_train, y_train)
# y_val_pred = classifier.predict(X_val)

In [ ]:
# # Commented as it didn't perform well
# from sklearn.ensemble import RandomForestClassifier
# classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
# classifier.fit(X_train, y_train)
# y_val_pred = classifier.predict(X_val)

In [ ]:
# # Commented as it didn't perform well
# from xgboost import XGBClassifier
# classifier = XGBClassifier()
# classifier.fit(X_train, y_train.ravel())
# y_val_pred = classifier.predict(X_val)

Now I tried cross-validation and tuned the parametes to get a better model

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'poly', 'sigmoid']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_) 
print(grid.best_estimator_)

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
y_val_pred = grid.predict(X_val)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_val, y_val_pred)) 
print(classification_report(y_val, y_val_pred))

0.8833461243284727
              precision    recall  f1-score   support

           0       0.67      0.50      0.57        12
           1       0.92      0.91      0.92       292
           2       0.83      0.79      0.81       135
           3       0.69      0.88      0.78        57
           4       0.88      0.64      0.74        22
           5       0.00      0.00      0.00         2
           6       0.88      0.90      0.89       252
           7       0.54      0.39      0.45        18
           8       0.90      0.41      0.56        22
           9       0.90      0.94      0.92       283
          10       0.94      0.95      0.95       208

    accuracy                           0.88      1303
   macro avg       0.74      0.66      0.69      1303
weighted avg       0.88      0.88      0.88      1303



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Now I rescaled the full training set (including the validation set) and transformed the test set based on that.

In [ ]:
# Feature Scaling to bring the variable in a single scale
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_full = sc.fit_transform(X_train_full)
X_test = sc.transform(X_test)

We'll now re-fit the final model with complete training set (including the validation set) and predict using the test set.

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0, C=10, gamma=0.01)
classifier.fit(X_train_full, y_train_full.ravel())
y_pred = classifier.predict(X_test)

Used StratifiedKFold for cross-validation and tested again.

In [ ]:
from sklearn.model_selection import StratifiedKFold

param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'poly']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, scoring='accuracy', cv=StratifiedKFold(n_splits=5))

grid.fit(X_train_full, y_train_full)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.224, total=   7.2s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.2s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.225, total=   7.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   14.2s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.225, total=   7.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.224, total=   7.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.224, total=   7.1s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.874, total=   1.5s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.858, total=   1.4s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.855, total=   1.4s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.871, total=   1.5s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] .

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed: 12.4min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [ ]:
print(grid.best_params_) 
print(grid.best_estimator_)

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
y_pred = grid.predict(X_test)

I inverse-transformed the label-encoding we did earlier.

In [ ]:
y_pred = labelencoder.inverse_transform(y_pred)

Saved the output in CSV file in GDrive

In [ ]:
output = pd.DataFrame(data={"Id": df_test["Id"], "Kingdom": y_pred})
output.to_csv(path_or_buf="/content/gdrive/My Drive/Codon-Hackathon-result.csv", index=False)

Below I have the models which helped me to increase the accuracy even more.

Here I utilised the ensemble method of Stacking. \
I stacked a KNN, SVM, NuSVM and a Logistic Regression Model (as combiner)  here. \
It performed pretty good

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold

level0 = list()
level0.append(('knn', KNeighborsClassifier()))
level0.append(('svm', SVC(kernel = 'rbf', C=10, gamma=0.01)))
level0.append(('NuSvm',NuSVC(kernel = "rbf", nu = 0.007)))
# define meta learner model
level1 = LogisticRegression(max_iter=10000)
# define the stacking ensemble
classifier = StackingClassifier(estimators=level0, final_estimator=level1, cv=StratifiedKFold(n_splits=5))
classifier.fit(X_train, y_train.ravel())
y_val_pred = classifier.predict(X_val)

Later, I tried a Multi-Layer Perceptron Classifier as One-VS-Rest Classifier. \
It performed even better.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
dnns_classifier = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes = [500]*5, random_state = 42))
dnns_classifier.fit(X_train, y_train)
y_val_pred = dnns_classifier.predict(X_val)

Here is my final stacked Classification Model where I stacked
* KNN
* SVM
* NuSVM
* Multi-Layer Perceptron
* Multi-Layer Perceptron as One-Vs-Rest

As the combiner, I used SVM and cross-validated and tuned the hyperparameters for the same.


In [ ]:
from mlxtend.classifier import StackingCVClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

classifier1 = KNeighborsClassifier()
classifier2 = SVC(kernel = 'rbf', C=10, gamma=0.01, probability = True)
classifier3 = NuSVC(kernel = "rbf", nu = 0.007, probability = True)
classifier4 = MLPClassifier(alpha = 0.1, hidden_layer_sizes = (300,300,300,300,300), max_iter = 2000, random_state = 42)
classifier5 = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes = [250]*5, random_state = 42))
sclf = StackingCVClassifier(classifiers = [classifier1, classifier2, classifier3, classifier4, classifier5],
                            shuffle = False,
                            use_probas = True,
                            cv = 5,
                            meta_classifier = SVC(probability = True))

params = {"meta_classifier__kernel": ["linear", "rbf", "poly"],
          "meta_classifier__C": [1, 2]}

grid = GridSearchCV(estimator = sclf, 
                    param_grid = params, 
                    cv = 5,
                    scoring = "accuracy",
                    verbose = 3,
                    n_jobs = 5)

grid.fit(X_train_full, y_train_full)
y_pred = grid.predict(X_test)

In [ ]:
y_pred = labelencoder.inverse_transform(y_pred)

In [ ]:
output = pd.DataFrame(data={"Id": df_test["Id"], "Kingdom": y_pred})
output.to_csv(path_or_buf="/content/gdrive/My Drive/Codon-Hackathon-result.csv", index=False)